#### Linear system of equations

Linear system of equations solves $b=Ax$ for $x$, $A\in \mathbf{R}^{m \times m}$, `square and full rank`, we can use Cholesky, LDLT, or LU with partial pivoting

#### Cholesky

For `positive definite` matrices, we take Cholesky factorization $$A=LL^T$$

Then

* solve $Ly=b$ using `forward` substitution
* solve $L^Tx=y$ using `back` substitution

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
np.set_printoptions(formatter={'float': '{: 0.4f}'.format})

plt.style.use('dark_background')
# color: https://matplotlib.org/stable/gallery/color/named_colors.htm

In [ ]:
def cholesky_factorization(A):
    m = A.shape[0]
    l_mat = A.copy().astype(float)

    for k in range(m):
        if l_mat[k, k] <= 0:
            raise ValueError('Matrix is not positive definite')

        # Follow the first step, iteratively apply to a smaller and smaller K
        l_mat[k+1:, k+1:] -= np.outer(l_mat[k+1:, k], l_mat[k+1:, k]) / l_mat[k, k]
        l_mat[k:, k] /= np.sqrt(l_mat[k, k])

    return np.tril(l_mat)

def forward_substitution(L, b):
    m, n = L.shape
    x = np.zeros(n)
    for i in range(n):
        x[i] = (b[i] - np.dot(L[i, :i], x[:i])) / L[i, i]
    return x

def back_substitution(R, b):
    m, n = R.shape
    x = np.zeros(n)
    for i in range(n - 1, -1, -1):
        x[i] = (b[i] - np.dot(R[i, i + 1:], x[i + 1:])) / R[i, i]
    return x

In [ ]:
np.random.seed(42)
m = 50

A = np.random.randn(m, m)
A_sym = A.T @ A
x = np.random.randn(m)
b = A_sym @ x
b_2 = A @ x
print(np.linalg.cond(A))

41.591235376905374


In [ ]:
try:
    L = cholesky_factorization(A_sym)
    y_ch = forward_substitution(L, b)
    x_ch = back_substitution(L.T, y_ch)
    print(np.linalg.norm(x_ch - x))
except Exception as e:
    print(e)

2.443400533449532e-13


#### LDLT

For nonsingular `symmetric` matrices, we can use LDLT factorization

* $A=LDL^T$
* Forward substitution for $Ly=b$
* Scaling for $Dz=y$
* Back substitution for $L^Tx=z$

In [ ]:
def ldlt_factorization(A):
    # Assume A is symmetric and invertible
    m = A.shape[0]
    l_mat = A.copy().astype(float)
    d_mat = np.zeros(m)

    for k in range(m):
        d_mat[k] = l_mat[k, k]
        if l_mat[k, k] == 0:
            raise ValueError('Matrix is singular')

        l_mat[k+1:, k+1:] -= np.outer(l_mat[k+1:, k], l_mat[k+1:, k]) / l_mat[k, k]
        l_mat[k:, k] /= l_mat[k, k]

    return np.tril(l_mat), np.diag(d_mat)

In [ ]:
try:
    l_mat, d_mat = ldlt_factorization(A_sym)
    y_ldlt = forward_substitution(l_mat, b)
    z_ldlt = y_ldlt / np.diag(d_mat)
    x_ldlt = back_substitution(l_mat.T, z_ldlt)

    print(np.linalg.norm(x_ldlt - x))
except Exception as e:
    print(e)

2.7300175676353613e-13


#### Generic matrices possible using Cholesky and LDLT

For well-conditioned generic square matrices $A$, using Cholesky and LDLT is possible by solving

$$A^TAx=A^Tb$$

##### Cholesky

In [ ]:
try:
    L = cholesky_factorization(A.T @ A)
    y_ch = forward_substitution(L, A.T @ b_2)
    x_ch = back_substitution(L.T, y_ch)
    print(np.linalg.norm(x_ch - x))
except Exception as e:
    print(e)

1.4616675373045904e-13


##### LDLT

In [ ]:
try:
    l_mat, d_mat = ldlt_factorization(A.T @ A)
    y_ldlt = forward_substitution(l_mat, A.T @ b_2)
    z_ldlt = y_ldlt / np.diag(d_mat)
    x_ldlt = back_substitution(l_mat.T, z_ldlt)

    print(np.linalg.norm(x_ldlt - x))
except Exception as e:
    print(e)

1.8680752430985427e-13


#### LU with partial pivoting

For generic square matrices, we first find $P$, $L$, $U$ such that $PA=LU$

Then, we rewrite $Ax=b$ as $LUx=Pb$ and

* solve $Ly=Pb$ using `forward` substitution
* solve $Ux=y$ using `back` substitution

In [ ]:
def lu_factorization(A):
    m, n = A.shape
    u_mat = A.copy().astype(float)
    l_mat = np.identity(m)
    p_mat = np.identity(m)

    for k in range(m-1):
        # Find pivot
        pivot = np.argmax(np.abs(u_mat[k:, k])) + k

        if pivot != k:
            # Swap rows in u, p, and l
            u_mat[[k, pivot], :] = u_mat[[pivot, k], :]
            p_mat[[k, pivot], :] = p_mat[[pivot, k], :]
            l_mat[[k, pivot], :k] = l_mat[[pivot, k], :k]

        for j in range(k + 1, m):
            l_mat[j, k] = u_mat[j, k] / u_mat[k, k]
            # Subtract multiply of kth row from jth row
            u_mat[j, k:] -= l_mat[j, k] * u_mat[k, k:]

    return p_mat, l_mat, u_mat

In [ ]:
p, l, u = lu_factorization(A)
y_lu = forward_substitution(l, p @ b_2)
x_lu = back_substitution(u, y_lu)

print(np.linalg.norm(x_lu - x))

2.5865046977186647e-14


We see that when $A$ is well-conditioned, all is good!

#### When the matrix is ill-conditioned

Computing $A^TA$ can amplify the ill-conditionedness with Cholesky and LDLT

In [ ]:
np.random.seed(42)

A_ill = np.random.randn(m, m)
A_ill[:, 0] = A_ill[:, -1] + 1e-8 * np.random.randn(m)
x = np.random.randn(m)
b_3 = A_ill @ x
print(np.linalg.cond(A_ill))

856008450.4491345


##### Cholesky

In [ ]:
try:
    L = cholesky_factorization(A_ill.T @ A_ill)
    y_ch = forward_substitution(L, A_ill.T @ b_3)
    x_ch = back_substitution(L.T, y_ch)
    print(np.linalg.norm(x_ch - x))
except Exception as e:
    print(e)

12.967923732845527


##### LDLT

In [ ]:
try:
    l_mat, d_mat = ldlt_factorization(A_ill.T @ A_ill)
    y_ldlt = forward_substitution(l_mat, A_ill.T @ b_3)
    z_ldlt = y_ldlt / np.diag(d_mat)
    x_ldlt = back_substitution(l_mat.T, z_ldlt)
    print(np.linalg.norm(x_ldlt - x))
except Exception as e:
    print(e)

26.631502576582093


##### LU with partial pivoting

In [ ]:
p, l, u = lu_factorization(A_ill)
y_lu = forward_substitution(l, p @ b_3)
x_lu = back_substitution(u, y_lu)

print(np.linalg.norm(x_lu - x))

7.86444716582733e-08


#### On stability of LU with partial pivoting

For LU with partial pivoting, a key metric for stability is whether amplification of the entries takes place during decomposition, or the growth factor

$$\rho = \frac{\max |u_{ij}|}{\max |a_{ij}|}$$

and we have the following theorem

Let $\tilde{P}, \tilde{L}, \tilde{U}$ be computed using LU with partial pivoting, then, they satisfy

$$\tilde{L}\tilde{U}=\tilde{P}A+\delta A, \frac{\|\delta A\|}{\|A\|}=O(\rho \epsilon_m)$$

We generally want the growth factor to be in the order of 1

Below is an example from NLA book where the growth factor is exponential $\rho=2^{m-1}$, where $m$ is the size of $A$

In [ ]:
# Example from NLA book
A = np.array([[1., 0, 0, 0, 1], [-1, 1, 0, 0, 1], [-1, -1, 1, 0, 1], [-1, -1, -1, 1, 1], [-1, -1, -1, -1, 1]])

p_mat, l_mat, u_mat = lu_factorization(A)

print('A:\n', A)
print('P:\n', p_mat)
print('L:\n', l_mat)
print('U:\n', u_mat)

print('PA:\n', p_mat @ A)
print('LU:\n', l_mat @ u_mat)

A:
 [[ 1.0000  0.0000  0.0000  0.0000  1.0000]
 [-1.0000  1.0000  0.0000  0.0000  1.0000]
 [-1.0000 -1.0000  1.0000  0.0000  1.0000]
 [-1.0000 -1.0000 -1.0000  1.0000  1.0000]
 [-1.0000 -1.0000 -1.0000 -1.0000  1.0000]]
P:
 [[ 1.0000  0.0000  0.0000  0.0000  0.0000]
 [ 0.0000  1.0000  0.0000  0.0000  0.0000]
 [ 0.0000  0.0000  1.0000  0.0000  0.0000]
 [ 0.0000  0.0000  0.0000  1.0000  0.0000]
 [ 0.0000  0.0000  0.0000  0.0000  1.0000]]
L:
 [[ 1.0000  0.0000  0.0000  0.0000  0.0000]
 [-1.0000  1.0000  0.0000  0.0000  0.0000]
 [-1.0000 -1.0000  1.0000  0.0000  0.0000]
 [-1.0000 -1.0000 -1.0000  1.0000  0.0000]
 [-1.0000 -1.0000 -1.0000 -1.0000  1.0000]]
U:
 [[ 1.0000  0.0000  0.0000  0.0000  1.0000]
 [ 0.0000  1.0000  0.0000  0.0000  2.0000]
 [ 0.0000  0.0000  1.0000  0.0000  4.0000]
 [ 0.0000  0.0000  0.0000  1.0000  8.0000]
 [ 0.0000  0.0000  0.0000  0.0000  16.0000]]
PA:
 [[ 1.0000  0.0000  0.0000  0.0000  1.0000]
 [-1.0000  1.0000  0.0000  0.0000  1.0000]
 [-1.0000 -1.0000  1.0000  0

However, from NLA book



> Gaussian elimination with partial pivoting is highly unstable for certain matrices. For instability to occur, however, the column spaces of $A$ must be `skewed in a very special fashion`, one that is exponentially rare in at least one class of random matrices. Decades of computational experience suggest that matrices whose column spaces are skewed in this fashion arise `very rarely` in applications.



#### Equations with structured sub-blocks

$$\begin{bmatrix}A_{11} & A_{12} \\ A_{21} & A_{22}\end{bmatrix}\begin{bmatrix}x_1  \\ x_2\end{bmatrix}=\begin{bmatrix}b_1  \\ b_2\end{bmatrix}$$

If $A_{11}$ has some structure (e.g., it is easy to compute its inverse), we can solve $x_2$ first

* Compute $A_{11}^{-1}$
* Form $A_{11}^{-1}A_{12}$ and $A_{11}^{-1}b$, plug $x_1=A_{11}^{-1}(b_1-A_{12}x_2)$ into second block equation
* Solve $(A_{22}-A_{21}A_{11}^{-1}A_{12})x_2=b_2-A_{21}A_{11}^{-1}b_1$ for $x_2$
* Solve $x_1$ from $x_1=A_{11}^{-1}(b_1-A_{12}x_2)$ for $x_1$

This is known as `block elimination`